In [1]:
import torch
import torch.nn.functional as F
import torchvision.models as models
from jupyterplot import ProgressPlot
from tqdm.notebook import tqdm

from Define import *
import NUS_WIDE as nus
import TenNet as TenNet
import PredictionModel as P
from TripletLossFunc import TripletLossFunc

In [2]:
getTenNetFromFile = True
getDecoderFromFile = False
test = False

In [3]:
if test or device == torch.device('cpu'):
    Number_Of_Images_Train = 100
    Number_Of_Images_Valid = 100
    N_Epochs = 1
    N_Epochs_Decoder = 1
    
#NUS_WIDE
train_data = nus.NUS_WIDE_Helper(nus.DataSetType.Train_81, Number_Of_Images_Train)
valid_data = nus.NUS_WIDE_Helper(nus.DataSetType.Test_81,  Number_Of_Images_Valid)
#test_data = NUS_WIDE_Helper(DataSetType.Test_81)

batch_size = BATCH_SIZE
train_loader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = torch.utils.data.DataLoader(valid_data, shuffle=True, batch_size=batch_size)
#test_loader = torch.utils.data.DataLoader(test_data, shuffle=True, batch_size=batch_size)

KeyboardInterrupt: 

In [ ]:
if test or device == torch.device('cpu'):
    image_model = models.vgg11(pretrained=True).to(device)
else:
    image_model = models.vgg16(pretrained=True).to(device)
    
#image_model = TenNet_Image().to(device)
tag_model = TenNet.TenNet_Tag(train_data.get_tag_list()).to(device)

optim = torch.optim.Adam([{'params' : image_model.parameters()}, {'params' : tag_model.parameters()}], lr=0.0001)
triplet_loss = TripletLossFunc(Margin_Distance)

n_epochs = N_Epochs
Lambda = 0.01
min_valid_loss = -1
ten_res = []

In [ ]:
if getTenNetFromFile:
    TenNet.getTenModel(tag_model, image_model, name = "SavedModelState/IT_model.ckpt")
else:

    pbar = tqdm(range(n_epochs))

    for e in pbar:
        
        loss_dis_train =  TenNet.train(image_model, tag_model, train_loader, triplet_loss, Lambda, optim)
        loss_dis_valid = TenNet.evalue(image_model, tag_model, valid_loader, triplet_loss, Lambda, optim, e, min_valid_loss, True) 
    
        print(f"epoch:{e}:")
        TenNet.output_loss_dis(f" 1-train dataset train model", loss_dis_train) 
        TenNet.output_loss_dis(f" 2-valid dataset evalue model", loss_dis_valid)
        min_valid_loss = loss_dis_valid[5]
  
        ten_res.append([loss_dis_train,loss_dis_valid])
        
        if (test or device == torch.device('cpu')) and e == 3:
            break
        

In [ ]:
res = ten_res
TenNet.printLossLog(res, n_epochs)
TenNet.printLossProgressPlot(res, n_epochs)
TenNet.printDistanceProgressPlot(res, n_epochs, train=True)
TenNet.printDistanceProgressPlot(res, n_epochs, train=False)

In [ ]:
for (x_images,y_tags) in train_loader:
    y_tags = y_tags.to(device)
    x_images = x_images.to(device)
    break

In [ ]:
batch_loss, output = P.compute_loss(x_images, y_tags, image_model, predict_model, loss_funk)
print(batch_loss)
loss = torch.nn.MSELoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5)
output = loss(input, target)
print(output)
print(torch.mean(output))batch_loss, output = P.compute_loss(x_images, y_tags, image_model, predict_model, loss_funk)
print(batch_loss)

In [ ]:
loss_funk = torch.nn.MSELoss(reduce=False, size_average=False)
#loss_funk = F.pairwise_distance

n_epochs = N_Epochs_Decoder
threshold = 0.5

In [ ]:
predict_model = P.TagDecoder(train_data.get_tag_num()).to(device)
optim = torch.optim.Adam(predict_model.parameters(), lr=0.001)
max_accuracy = -1
predict_res = []

if getDecoderFromFile:
    P.getDecoderModel(predict_model)
else:
    pbar = tqdm(range(n_epochs))

    for e in pbar:
    
        loss_num_train = P.train(predict_model, tag_model, image_model, train_loader, loss_funk, optim, threshold, mod=P.LossModel.PredictModel)
        P.output_loss_num(f"epoch:{e}: 1-train dataset with train model", loss_num_train)
        
        loss_num_valid = P.predict(predict_model, tag_model, image_model, valid_loader, loss_funk, optim, threshold, True, max_accuracy, e)   
        P.output_loss_num(f"epoch:{e}: 2-valid dataset with evalue model", loss_num_valid)
        max_accuracy = loss_num_valid[5]
    
        predict_res.append([loss_num_train,loss_num_valid])

In [ ]:
res = predict_res
P.printLossLog(res, n_epochs)
P.printLossProgressPlot(res, n_epochs)
P.printTagNumProgressPlot(res, n_epochs)
P.printAccuracyProgressPlot(res, n_epochs)

In [ ]:
decoder_model = P.TagDecoder(train_data.get_tag_num()).to(device)
optim = torch.optim.Adam(decoder_model.parameters(), lr=0.001)
max_accuracy = -1
decoder_res = []
if getDecoderFromFile:
    P.getDecoderModel(decoder_model)
else:
    pbar = tqdm(range(n_epochs))

    for e in pbar:
    
        loss_num_train = P.train(decoder_model, tag_model, image_model, train_loader, loss_funk, optim, threshold, mod=P.LossModel.DecoderModel)
        P.output_loss_num(f"epoch:{e}: 1-train dataset with train model", loss_num_train)
        
        loss_num_valid = P.predict(decoder_model, tag_model, image_model, valid_loader, loss_funk, optim, threshold, True, max_accuracy, e)   
        P.output_loss_num(f"epoch:{e}: 2-valid dataset with evalue model", loss_num_valid)
        max_accuracy = loss_num_valid[5]
    
        decoder_res.append([loss_num_train,loss_num_valid])

In [ ]:
res = decoder_res
P.printLossLog(res, n_epochs)
P.printLossProgressPlot(res, n_epochs)
P.printTagNumProgressPlot(res, n_epochs)
P.printAccuracyProgressPlot(res, n_epochs)

In [4]:
str(Margin_Distance)

'9000'

In [ ]:
  
        '''
        print(f"epoch:{e}:")
        print("1-7 use the same weights, since just 1 updates the weights.\n")
        loss_dis_valid = TenNet.evalue(image_model, tag_model, valid_loader, triplet_loss, Lambda, optim, e, min_valid_loss, True)   
        TenNet.output_loss_dis(f" 2-valid dataset with evalue model", loss_dis_valid)
        min_valid_loss = loss_dis_valid[5]
        
        loss_dis_train = TenNet.evalue(image_model, tag_model, train_loader, triplet_loss, Lambda, optim, e, min_valid_loss, True)   
        TenNet.output_loss_dis(f" 3-train dataset with evalue model", loss_dis_train)
        
        print("2-3 use the same way to evalue the model (both eval() and no_grad()), but different data sets.\n")
        
        loss_dis_train = TenNet.evalue2(image_model, tag_model, train_loader, triplet_loss, Lambda, optim)   
        TenNet.output_loss_dis(f" 4-train dataset with evalue model, just evalue model", loss_dis_train)
        
        loss_dis_train = TenNet.evalue3(image_model, tag_model, train_loader, triplet_loss, Lambda, optim)   
        TenNet.output_loss_dis(f" 5-train dataset with evalue model, just no gradient", loss_dis_train)
        
        print("4-5 use the same data set, but different ways to train the model.One uses eval(), one uses no_grad().\n")
        
        loss_dis_train =  TenNet.train(image_model, tag_model, train_loader, triplet_loss, Lambda, optim, False)
        TenNet.output_loss_dis(f" 6-train dataset with train model, but doesn't updata", loss_dis_train)
        
        loss_dis_train =  TenNet.train(image_model, tag_model, train_loader, triplet_loss, Lambda, optim, False)
        TenNet.output_loss_dis(f" 6-train dataset with train model, but doesn't updata", loss_dis_train)
        
        loss_dis_train =  TenNet.train(image_model, tag_model, train_loader, triplet_loss, Lambda, optim)
        TenNet.output_loss_dis(f" 1-train dataset with train model", loss_dis_train)
        
        print("6-6-1 use the same data set and way to train the model, but 6 doesn't updata the weight.\n")
        '''